# AutoGen Team
A team is a group of agents that work together to achieve a common goal.


## Load Azure Configurations

In [21]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client
Using the model client class

In [2]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)

## Creating a Team

RoundRobinGroupChat is a team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.

We create a team with two AssistantAgent and a TextMentionTermination condition that stops the team when a specific word is detected in the agent’s response.

In [3]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient


# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=az_model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=az_model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

## Running a Team
Call the run() method to start the team with a task.

The termination condition was met when the word “APPROVE” is detected in the agent’s response. When the team stops, it returns a TaskResult object with all the messages produced by the agents in the team.

In [ ]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
result = await team.run(task="Write a short poem about the fall season.")
print(result)

## Observing a Team
Similar to the agent’s on_messages_stream() method, you can stream the team’s messages while it is running by calling the run_stream() method. This method returns a generator that yields messages produced by the agents in the team as they are generated, with the final item being the TaskResult object.

In [18]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None content='Write a short poem about the fall season.' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=28, completion_tokens=102) content='Leaves descend in golden showers,  \nWhispering tales of autumn hours.  \nCrisp air dances through the trees,  \nCarving paths for wandering breeze.  \n\nPumpkins rest on fields once green,  \nHints of spice in every scene.  \nSweaters wrap us in their care,  \nAs autumn whispers fill the air.  \n\nThe world slows down, a gentle pause,  \nNature’s grace commands applause.  \nIn this fleeting, vibrant scope,  \nFall enchants with warmth and hope.  ' type='TextMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=148, completion_tokens=202) content='The poem beautifully captures the essence of the fall season with vivid imagery and a soothing rhythm. The imagery of "leaves descend in golden showers" and "pumpkins rest on fields once green" effectively paints a picture of the season\'s

## Console()
The Console() method provides a convenient way to print messages to the console with proper formatting.

In [19]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a short poem about the fall season."))  # Stream the messages to the console.

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Leaves cascade in crimson and gold,  
Whispers of autumn's stories unfold.  
A cool breeze carries the scent of earth,  
As nature celebrates its yearly rebirth.  

Pumpkins dot fields, round and bright,  
While days offer a softer light.  
Sweaters emerge from cedar-filled drawers,  
As apples are gathered for cider, and s'mores.  

The world prepares for a slower pace,  
In the gentle glow of autumn's embrace.  
Silent as a deep, serene breath,  
Fall is life in quiet rest.
[Prompt tokens: 28, Completion tokens: 111]
---------- critic ----------
This is a beautifully crafted poem that captures the essence of the fall season wonderfully. The imagery of "leaves cascade in crimson and gold" paints a vivid picture of autumn's colorful landscape. The use of sensory elements, such as the "cool breeze" and the "scent of earth," helps to create a strong connection with the reader's senses, allo

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=111), content="Leaves cascade in crimson and gold,  \nWhispers of autumn's stories unfold.  \nA cool breeze carries the scent of earth,  \nAs nature celebrates its yearly rebirth.  \n\nPumpkins dot fields, round and bright,  \nWhile days offer a softer light.  \nSweaters emerge from cedar-filled drawers,  \nAs apples are gathered for cider, and s'mores.  \n\nThe world prepares for a slower pace,  \nIn the gentle glow of autumn's embrace.  \nSilent as a deep, serene breath,  \nFall is life in quiet rest.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=157, completion_tokens=279), content='This is a beautifully crafted poem that captures the essence of the fall season wonderfully. The imagery of "leaves cascade in c

## Team has a State

In [20]:
await Console(team.run_stream(task="What was the last message?"))  # Stream the messages to the console.

---------- user ----------
What was the last message?
---------- primary ----------
The last message I provided was a response to your feedback on the poem. I expressed gratitude for your thoughtful critique and offered to help further if you're interested in exploring additional aspects of autumn or trying out different poetic styles. Let me know if there's anything else you'd like to explore or if you have any other requests!
[Prompt tokens: 528, Completion tokens: 62]
---------- critic ----------
I apologize for any confusion. My previous message was simply approving your response, acknowledging that you addressed the feedback constructively. If there's anything more specific you'd like to discuss regarding your poem or any other creative endeavors, please feel free to share!
[Prompt tokens: 608, Completion tokens: 48]
---------- primary ----------
No worries at all! I'm glad the response was helpful. If you have any further thoughts, questions, or areas you'd like to delve into—whe

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What was the last message?', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=528, completion_tokens=62), content="The last message I provided was a response to your feedback on the poem. I expressed gratitude for your thoughtful critique and offered to help further if you're interested in exploring additional aspects of autumn or trying out different poetic styles. Let me know if there's anything else you'd like to explore or if you have any other requests!", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=608, completion_tokens=48), content="I apologize for any confusion. My previous message was simply approving your response, acknowledging that you addressed the feedback constructively. If there's anything more specific you'd like to discuss regarding your poem or any other creative endeavors, please feel free to share!", typ

## Resetting a Team
You can reset the team by calling the reset() method. This method will clear the team’s state, including all agents. It will call the each agent’s on_reset() met

In [12]:
await team.reset()  # Reset the team for the next run.

## Stopping a Team
You can also stop the team from outside by using the ExternalTermination.

Calling set() on ExternalTermination will stop the team when the current agent’s turn is over. 
Thus, the team may not stop immediately. This allows the current agent to finish its turn and broadcast the final message to the team before the team stops, keeping the team’s state consistent.

In [ ]:
# Create a new team with an external termination condition.
external_termination = ExternalTermination()
team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=external_termination | text_termination,  # Use the bitwise OR operator to combine conditions.
)

# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))

# Wait for some time.
await asyncio.sleep(0.1)

# Stop the team.
external_termination.set()

# Wait for the team to finish.
await run # Primary agent is still allowed to complete its task.

## Resuming a Team
Teams are stateful and maintains the conversation history and context after each run, unless you reset the team.

You can resume a team to continue from where it left off by calling the run() or run_stream() method again without a new task. RoundRobinGroupChat will continue from the next agent in the round-robin order.

In this example, critic agent continued where primary agent left off

In [ ]:
await Console(team.run_stream())  # Resume the team to continue the last task.

## Aborting a Team
You can abort a call to run() or run_stream() during execution by setting a CancellationToken passed to the cancellation_token parameter.

Different from stopping a team, aborting a team will immediately stop the team and raise a CancelledError exception.

In [17]:
# Create a cancellation token.
cancellation_token = CancellationToken()

# Use another coroutine to run the team.
run = asyncio.create_task(
    team.run(
        task="Translate the poem to Spanish.",
        cancellation_token=cancellation_token,
    )
)

# Cancel the run.
cancellation_token.cancel()

try:
    result = await run  # This will raise a CancelledError.
except asyncio.CancelledError:
    print("Task was cancelled.")

Task was cancelled.
